In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer#,TfidfTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os

In [ ]:
#ad_feature=pd.read_csv('../data/adFeature.csv')
#user_feature=pd.read_csv('../data/userFeature/userFeature_1.csv')#29
#train=pd.read_csv('../data/train/train_1.csv')
#predict=pd.read_csv('../data/test1/test1_1.csv')

In [2]:
data=pd.read_csv('../data/newdata01.csv')

In [3]:
#train.loc[train['label']==-1,'label']=0
#predict['label']=-1
#data=pd.concat([train,predict])
#data=pd.merge(data,ad_feature,on='aid',how='left')
#data=pd.merge(data,user_feature,on='uid',how='left')
data=data.fillna('-1')

In [ ]:
#转化数值类型，减少内存
#obj2uint8=['label','advertiserId','campaignId','creativeId','creativeSize','adCategoryId','productId','productType','age','gender','education','consumptionAbility','carrier','house']
#for feature in obj2uint8:
#    #print(feature+' is transformed to interger num...')
#    data[feature]=data[feature].astype('uint8')

##print('LBS is transformed to interger num...')
#data['LBS']=data['LBS'].astype('uint16')
#
#print('data has been reshaped OK!')

#——————————-下面对多值的特征进行处理————————————————

In [ ]:
# 处理联网类型特征,多值->对应位标记为1

#因为只读了部分userFeature,所以做连接没连到的值会变成'-1'，这里暂且先替换为'0'。   
#遍历全部userFeature连接后是不会有'-1'的,就可以删掉下面这行
data.loc[ (data.ct == '-1' ), 'ct' ] = '0'

ct = data['ct'].values
ct = [m.split(' ') for m in ct]
cts = []
for i in ct:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    cts.append(index)
    
for i in ['0','1','2','3','4']:
    name='ct_'+ i
    locals()[name]= []
    for line in cts:
        c =  np.array(line)
        locals()[name].append(c[0+int(i)])
    data[name]=locals()[name]

data = data.drop(['ct'], axis=1)
print('ct has been transformed OK!')

In [ ]:
# 处理操作系统类型特征,多值->对应位标记为1

#因为只读了部分userFeature,所以做连接没连到的值会变成'-1'，这里暂且先替换为'0'。   
#遍历全部userFeature连接后是不会有'-1'的,就可以删掉下面这行
data.loc[ (data.os == '-1' ), 'os' ] = '0'

os = data['os'].values
os = [m.split(' ') for m in os]
oss = []
for i in os:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    oss.append(index)
    
for i in ['0','1','2']:
    name='os_'+ i
    locals()[name]= []
    for line in oss:
        o =  np.array(line)
        locals()[name].append(o[0+int(i)])
    data[name]=locals()[name]

data = data.drop(['os'], axis=1)
print('os has been transformed OK!')

In [ ]:
#app安装&活跃特征、结婚相关状态特征
#data['has_appins'] = data['appIdInstall'].apply(lambda x: 0 if ((not x.strip()) or str(x)=='-1') else 1)
#data = data.drop(['appIdInstall'], axis=1)

#data['has_appact'] = data['appIdAction'].apply(lambda x: 0 if ((not x.strip()) or str(x)=='-1') else 1) 
#data = data.drop(['appIdAction'], axis=1)

#data['max_marriageStatus'] = data['marriageStatus'].apply(lambda x: 0 if ((not str(x).strip()) or str(x)=='-1') else max(int(i) for i in str(x).split(' ')))
#data = data.drop(['marriageStatus'], axis=1)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054903 entries, 0 to 1054902
Data columns (total 33 columns):
aid                   1054903 non-null int64
uid                   1054903 non-null int64
label                 1054903 non-null int64
advertiserId          1054903 non-null int64
campaignId            1054903 non-null int64
creativeId            1054903 non-null int64
creativeSize          1054903 non-null int64
adCategoryId          1054903 non-null int64
productId             1054903 non-null int64
productType           1054903 non-null int64
age                   1054903 non-null int64
gender                1054903 non-null int64
marriageStatus        1054903 non-null object
education             1054903 non-null int64
consumptionAbility    1054903 non-null int64
LBS                   1054903 non-null float64
interest1             1054903 non-null object
interest2             1054903 non-null object
interest3             1054903 non-null object
interest4             1054

#------------------------------分割线（底下的兴趣特征还没弄完）-----------------------------------------

In [ ]:
'''
train_all_interest = X_train[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
test_all_interest = X_test[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
num_all_interest_train = []
num_all_interest_test = []
aids = []
train_array = np.array(train_all_interest)
test_array = np.array(test_all_interest)

for i in range(train_array.shape[0]):
    num = 0
    inter = train_array[i]
    for j in inter:
        inter_lis = j.split(' ')
        if inter_lis[0] == '-1':
            continue
        num += len(inter_lis)
    num_all_interest_train.append(num)
'''

In [ ]:
'''
data.info()
'''

——————————-下面是统计特征————————————————

In [ ]:
# 增加不同的年龄段的用户数量
#num_ad2age = train['uid'].groupby('age').uid.nunique().value_counts()
#num_ad2age = pd.DataFrame({
#    'age': num_ad2age.index,
#    'num_ad2age' : num_ad2age.values
#})

#data = pd.merge(data, num_ad2age, on=['uid'], how='left')
#print('加入广告推送给不同用户数量特征,OK!')
#print(num_advertise_touser)

In [ ]:
# 将上面新加入的特征进行归一化
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#scaler.fit(data[['num_ad2age']].values)
#data[['num_ad2age']] = scaler.transform(data[['num_ad2age']].values)

#print('统计特征的归一化,OK!')

In [ ]:
#data.info()

#------------------------------分割线(模型训练)-----------------------------------------

In [5]:
one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','house']
vector_feature=['appIdAction','appIdInstall','marriageStatus','interest1','interest2','interest3','interest4','interest5','kw1','kw2','kw3','topic1','topic2','topic3']
#LabelEncoder将各种标签分配一个可数的连续编号
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])
        
train=data[data.label!=-1]
train_y=train.pop('label')

test=data[data.label==-1]
res=test[['aid','uid']]
test=test.drop('label',axis=1)

train_x=train[['advertiserId','campaignId', 'creativeId','creativeSize','adCategoryId', 'productId','productType'
               ,'ct_0','ct_1','ct_2','ct_3','ct_4','os_0','os_1','os_2']]
              # ,'has_appins','has_appact','max_marriageStatus']]
test_x = test[['advertiserId','campaignId', 'creativeId','creativeSize','adCategoryId', 'productId', 'productType'
              ,'ct_0','ct_1','ct_2','ct_3','ct_4','os_0','os_1','os_2']]
              #,'has_appins','has_appact','max_marriageStatus']]


In [6]:
#one-hot特征reshape
enc = OneHotEncoder()
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
print('one-hot prepared !')
del train_a
del test_a

one-hot prepared !


In [7]:
##向量特征reshape
cv=CountVectorizer(ngram_range=(1, 2),token_pattern='(?u)\\b\\w+\\b')
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
print('cv prepared !')
del train_a
del test_a

cv prepared !


In [ ]:
#sparse.save_npz('all_features_train',train_x)
#sparse.save_npz('all_features_test',test_x)

In [8]:
def LGB_predict(train_x,train_y,test_x,res):
    print("LGB test")
    clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,zero_as_missing=True,
        max_depth=-1, n_estimators=500, objective='binary',
        subsample=0.9, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.1, min_child_weight=50, random_state=2018, n_jobs=100
    )
    clf.fit(train_x, train_y, eval_set=[(train_x, train_y)], eval_metric='auc',early_stopping_rounds=100)
    res['score'] = clf.predict_proba(test_x)[:,1]
    res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
    res.to_csv('../data/submission.csv', index=False)
    os.system('zip baseline.zip ../data/submission.csv')
    return clf

model=LGB_predict(train_x,train_y,test_x,res)

LGB test
[1]	valid_0's auc: 0.646316
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.657202
[3]	valid_0's auc: 0.661393
[4]	valid_0's auc: 0.662702
[5]	valid_0's auc: 0.665381
[6]	valid_0's auc: 0.667575
[7]	valid_0's auc: 0.668998
[8]	valid_0's auc: 0.670869
[9]	valid_0's auc: 0.672537
[10]	valid_0's auc: 0.673724
[11]	valid_0's auc: 0.674416
[12]	valid_0's auc: 0.676249
[13]	valid_0's auc: 0.676706
[14]	valid_0's auc: 0.677309
[15]	valid_0's auc: 0.678744
[16]	valid_0's auc: 0.679559
[17]	valid_0's auc: 0.68087
[18]	valid_0's auc: 0.681717
[19]	valid_0's auc: 0.683106
[20]	valid_0's auc: 0.683913
[21]	valid_0's auc: 0.68486
[22]	valid_0's auc: 0.685795
[23]	valid_0's auc: 0.686529
[24]	valid_0's auc: 0.687394
[25]	valid_0's auc: 0.688635
[26]	valid_0's auc: 0.689471
[27]	valid_0's auc: 0.690268
[28]	valid_0's auc: 0.691135
[29]	valid_0's auc: 0.692484
[30]	valid_0's auc: 0.693389
[31]	valid_0's auc: 0.694118
[32]	valid_0's auc: 0.694949
[33]	valid_

[277]	valid_0's auc: 0.736272
[278]	valid_0's auc: 0.736336
[279]	valid_0's auc: 0.736402
[280]	valid_0's auc: 0.73645
[281]	valid_0's auc: 0.7365
[282]	valid_0's auc: 0.736575
[283]	valid_0's auc: 0.736626
[284]	valid_0's auc: 0.736684
[285]	valid_0's auc: 0.736722
[286]	valid_0's auc: 0.73678
[287]	valid_0's auc: 0.736839
[288]	valid_0's auc: 0.737006
[289]	valid_0's auc: 0.737144
[290]	valid_0's auc: 0.737204
[291]	valid_0's auc: 0.737263
[292]	valid_0's auc: 0.737324
[293]	valid_0's auc: 0.737382
[294]	valid_0's auc: 0.73746
[295]	valid_0's auc: 0.7375
[296]	valid_0's auc: 0.737547
[297]	valid_0's auc: 0.737613
[298]	valid_0's auc: 0.737661
[299]	valid_0's auc: 0.737713
[300]	valid_0's auc: 0.737756
[301]	valid_0's auc: 0.7378
[302]	valid_0's auc: 0.737909
[303]	valid_0's auc: 0.737982
[304]	valid_0's auc: 0.738028
[305]	valid_0's auc: 0.738081
[306]	valid_0's auc: 0.738142
[307]	valid_0's auc: 0.738187
[308]	valid_0's auc: 0.738228
[309]	valid_0's auc: 0.738279
[310]	valid_0's auc

C:\Users\caijiayue\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [ ]:
#不做任何处理，    得分：0.747515
#加上ct、os的处理，得分：0.747254